In [15]:
import random
import time
import heapq
from itertools import combinations

# Generator & evaluators
def generate_k_sat(k, m, n):
    if k > n:
        raise ValueError("k must be <= n")
    formula = []
    variables = list(range(1, n + 1))
    for _ in range(m):
        clause_vars = random.sample(variables, k)  # distinct variables
        clause = []
        for var in clause_vars:
            literal = var if random.choice([True, False]) else -var
            clause.append(literal)
        formula.append(clause)
    return formula

def eval_clause(clause, assignment):
    for lit in clause:
        var = abs(lit)
        val = assignment.get(var, False)
        if (lit > 0 and val) or (lit < 0 and not val):
            return True
    return False

# Return number of satisfied clauses

def eval_formula(formula, assignment):
    return sum(1 for c in formula if eval_clause(c, assignment))

# cost: number of unsatisfied clauses (lower is better)

def unsatisfied_count(formula, assignment):
    return len(formula) - eval_formula(formula, assignment)

# Reward: total number of satisfied literals (higher is better)
def satisfied_literals_count(formula, assignment):
    total = 0
    for c in formula:
        for lit in c:
            var = abs(lit)
            val = assignment.get(var, False)
            if (lit > 0 and val) or (lit < 0 and not val):
                total += 1
    return total

# Wrapper: produce a cost function (lower is better) for any heuristic
def make_cost_fn(name):
    if name == 'unsat_clauses':
        return lambda f,a: unsatisfied_count(f,a)
    elif name == 'satisfied_literals':
        # convert reward -> cost by negating
        return lambda f,a: -satisfied_literals_count(f,a)
    else:
        raise ValueError("unknown heuristic")

def random_assignment(n):
    return {v: random.choice([True, False]) for v in range(1, n + 1)}

# Heuristic 1: count unsatisfied clauses
def heuristic_1(formula, assignment):
    unsat = 0
    for c in formula:
        if not eval_clause(c, assignment):
            unsat += 1
    return unsat

# Heuristic 2: number of satisfied literals in formula
def heuristic_2(formula, assignment):
    total = 0
    for c in formula:
        for lit in c:
            var = abs(lit)
            val = assignment.get(var, False)
            if (lit > 0 and val) or (lit < 0 and not val):
                total += 1
    return total


# Hill-climbing (with restarts)
def hill_climbing(formula, n, cost_fn, max_iterations=1000):
    current = random_assignment(n)
    current_cost = cost_fn(formula, current)
    for _ in range(max_iterations):
        if current_cost == 0:  # solved (no unsatisfied clauses)
            break
        improved = False
        # Try all single-bit flips, pick best (greedy)
        best_neighbor = None
        best_cost = current_cost
        for var in range(1, n + 1):
            neighbor = current.copy()
            neighbor[var] = not neighbor[var]
            cost = cost_fn(formula, neighbor)
            if cost < best_cost:
                best_cost = cost
                best_neighbor = neighbor
        if best_neighbor is not None:
            current = best_neighbor
            current_cost = best_cost
            improved = True
        if not improved:
            break
    return current, current_cost

def hill_climbing_restarts(formula, n, cost_fn, restarts=10, max_iter=1000):
    best = None
    best_cost = float('inf')
    times = []
    start = time.time()
    for _ in range(restarts):
        a, c = hill_climbing(formula, n, cost_fn, max_iter)
        if c < best_cost:
            best_cost = c
            best = a
        if best_cost == 0:
            break
    elapsed = time.time() - start
    return best, best_cost, elapsed

# Beam search (fixed direction: lower is better cost)
def beam_search(formula, n, cost_fn, beam_width=3, max_iterations=100):
    # Initialize beam: build consistent (cost, assignment) tuples
    beam = []
    for _ in range(beam_width):
        a = random_assignment(n)
        beam.append((cost_fn(formula, a), a))
    for _ in range(max_iterations):
        # check if solved
        for cost, assignment in beam:
            if cost == 0:
                return assignment, cost
        new_beam = []
        for cost, assignment in beam:
            for var in range(1, n + 1):
                neighbor = assignment.copy()
                neighbor[var] = not neighbor[var]
                new_cost = cost_fn(formula, neighbor)
                new_beam.append((new_cost, neighbor))
        # keep best beam_width (smallest cost)
        beam = heapq.nsmallest(beam_width, new_beam, key=lambda x: x[0])
        if not beam:
            break
    return beam[0][1], beam[0][0]

# VND
def neighborhoods(n):
    ones = [[v] for v in range(1, n + 1)]
    twos = [list(c) for c in combinations(range(1, n + 1), 2)]
    threes = [list(c) for c in combinations(range(1, n + 1), 3)]
    return [ones, twos, threes]

def vnd(formula, n, cost_fn, max_iterations=1000):
    current = random_assignment(n)
    current_cost = cost_fn(formula, current)
    neighs = neighborhoods(n)
    k = 0
    while k < len(neighs) and current_cost > 0:
        improved = False
        for move_vars in neighs[k]:
            neighbor = current.copy()
            for var in move_vars:
                neighbor[var] = not neighbor[var]
            cost = cost_fn(formula, neighbor)
            if cost < current_cost:
                current, current_cost = neighbor, cost
                improved = True
                break
        if improved:
            k = 0
        else:
            k += 1
    return current, current_cost

def vnd_restarts(formula, n, cost_fn, restarts=5):
    best = None
    best_cost = float('inf')
    start = time.time()
    for _ in range(restarts):
        a, c = vnd(formula, n, cost_fn)
        if c < best_cost:
            best_cost = c
            best = a
        if best_cost == 0:
            break
    elapsed = time.time() - start
    return best, best_cost, elapsed

# Experiment harness (penetrance)
def run_single_instance_and_method(formula, n, method_name, cost_fn):
    if method_name == 'Hill Climbing':
        a, c, t = hill_climbing_restarts(formula, n, cost_fn, restarts=10, max_iter=1000)
    elif method_name == 'Beam-3':
        start = time.time()
        a, c = beam_search(formula, n, cost_fn, beam_width=3, max_iterations=200)
        t = time.time() - start
    elif method_name == 'Beam-4':
        start = time.time()
        a, c = beam_search(formula, n, cost_fn, beam_width=4, max_iterations=200)
        t = time.time() - start
    elif method_name == 'VND':
        a, c, t = vnd_restarts(formula, n, cost_fn, restarts=5)
    else:
        raise ValueError("unknown method")
    # resolved if no unsatisfied clauses:
    solved = (unsatisfied_count(formula, a) == 0)
    return solved, c, t

def run_experiment(k, m, n, method_name, heuristic_name, instances=50, seed=None):
    if seed is not None:
        random.seed(seed)
    cost_fn = make_cost_fn(heuristic_name)
    solved_count = 0
    total_time = 0.0
    total_cost = 0.0
    for i in range(instances):
        formula = generate_k_sat(k, m, n)
        solved, cost, t = run_single_instance_and_method(formula, n, method_name, cost_fn)
        if solved:
            solved_count += 1
        total_time += t
        total_cost += cost
    penetrance = solved_count / instances
    return {
        'method': method_name,
        'heuristic': heuristic_name,
        'k': k, 'm': m, 'n': n,
        'instances': instances,
        'penetrance': penetrance,
        'avg_time_sec': total_time / instances,
        'avg_cost': total_cost / instances
    }



### Run experiment harness comparing methods and heuristics on slightly larger instances

In [ ]:
import pandas as pd

results_list = []

for method_name in ['Hill Climbing', 'Beam-3', 'Beam-4', 'VND']:
    for heuristic_name in ['unsat_clauses', 'satisfied_literals']:
        result = run_experiment(
            k=3,
            m=40,
            n=20,
            method_name=method_name,
            heuristic_name=heuristic_name,
            instances=10,
            seed=123
        )
        results_list.append(result)  
        
results = pd.DataFrame(results_list)
print("\n=== Experiment Summary ===")
print(results)



=== Experiment Summary ===
          method           heuristic  k   m   n  instances  penetrance  \
0  Hill Climbing       unsat_clauses  3  40  20         10         1.0   
1  Hill Climbing  satisfied_literals  3  40  20         10         0.3   
2         Beam-3       unsat_clauses  3  40  20         10         1.0   
3         Beam-3  satisfied_literals  3  40  20         10         0.0   
4         Beam-4       unsat_clauses  3  40  20         10         1.0   
5         Beam-4  satisfied_literals  3  40  20         10         0.1   
6            VND       unsat_clauses  3  40  20         10         1.0   
7            VND  satisfied_literals  3  40  20         10         0.0   

   avg_time_sec  avg_cost  
0      0.001237       0.0  
1      0.016924     -79.6  
2      0.001447       0.0  
3      0.115435     -79.4  
4      0.002203       0.0  
5      0.149407     -79.4  
6      0.000600       0.0  
7      0.000600     -67.8  


In [17]:
results

,method,heuristic,k,m,n,instances,penetrance,avg_time_sec,avg_cost
0,Hill Climbing,unsat_clauses,3,40,20,10,1.0,0.001237,0.0
1,Hill Climbing,satisfied_literals,3,40,20,10,0.3,0.016924,-79.6
2,Beam-3,unsat_clauses,3,40,20,10,1.0,0.001447,0.0
3,Beam-3,satisfied_literals,3,40,20,10,0.0,0.115435,-79.4
4,Beam-4,unsat_clauses,3,40,20,10,1.0,0.002203,0.0
5,Beam-4,satisfied_literals,3,40,20,10,0.1,0.149407,-79.4
6,VND,unsat_clauses,3,40,20,10,1.0,0.000600,0.0
7,VND,satisfied_literals,3,40,20,10,0.0,0.000600,-67.8
